# Large Gridded Exports from GEE

This notebook shows how to export a country-scale raster from Earth Engine as separate tiles using a grid. The key is to ensure use of `crs` and `crsTransform` to ensure all the tiles are in the same pixel grid and align correctly with the target projection.

[Read the full post](https://spatialthoughts.com/2024/10/23/large-image-exports-gee/)

In [ ]:
import geemap
import ee

#### Initialization

First of all, you need to run the following cells to initialize the API and authorize your account. You must have a Google Cloud Project associated with your GEE account. Replace the `cloud_project` with your own project from [Google Cloud Console](https://console.cloud.google.com/).

In [ ]:
# Replace the cloud_project with your own project
cloud_project = 'spatialthoughts'
try:
    ee.Initialize(project=cloud_project)
except:
    ee.Authenticate()
    ee.Initialize(project=cloud_project)

#### Data Prep

We select a country and create a clipped ESA WorldCover 2020 classification for the region.

In [ ]:
worldcover = ee.ImageCollection("ESA/WorldCover/v100")
lsib = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017")

# Select the country
country = lsib.filter(ee.Filter.eq('country_na', 'Estonia'))
geometry = country.geometry()

# Select the image for export
image = worldcover.first().clip(geometry)

m = geemap.Map(width=800)
m.addLayer(image, {}, 'Input Image');
m.centerObject(country, 7)
m

#### Create a Grid

We create a grid and calculate the parameters for the CRS Transform. Each tile of the grid will be exported as a separate image on the chosen pixel grid.

In [ ]:
# Choose the export CRS
crs = 'EPSG:3301'

# Choose the pixel size for export (meters)
pixelSize = 10

# Choose the export tile size (pixels)
tileSize = 10000

# Calculate the grid size (meters)
gridSize = tileSize * pixelSize

# Create the grid covering the geometry bounds
bounds = geometry.bounds(**{
  'proj': crs, 'maxError': 1
})

grid = bounds.coveringGrid(**{
  'proj':crs, 'scale': gridSize
})

m.addLayer(grid, {'color': 'blue'}, 'Grid')
m

#### Calculate the CRS Transform

In [ ]:
# Calculate the coordinates of the top-left corner of the grid
bounds = grid.geometry().bounds(**{
  'proj': crs, 'maxError': 1
});

# Extract the coordinates of the grid
coordList = ee.Array.cat(bounds.coordinates(), 1)

xCoords = coordList.slice(1, 0, 1)
yCoords = coordList.slice(1, 1, 2)

# We need the coordinates of the top-left pixel
xMin = xCoords.reduce('min', [0]).get([0,0])
yMax = yCoords.reduce('max', [0]).get([0,0])

# Create the CRS Transform

# The transform consists of 6 parameters:
# [xScale, xShearing, xTranslation,
#  yShearing, yScale, yTranslation]
transform = ee.List([
    pixelSize, 0, xMin, 0, -pixelSize, yMax]).getInfo()
print(transform)

#### Resample or Aggregate Pixels

By default, the images are resampled to the target pixel grid using the Nearest Neighbor method. This is fine for most types of images, but you may want to change this behavior for certain types of operations. For discrete rasters, such as landcover classification, nearest neighbor is appropriate. For climate or elevation rasters, you may want to enable `bilinear` or `bicubic` interpolation.



In [ ]:
# Uncomment below to enable resampling
# This is not required for classification images
# image = image.resample('bicubic')

#### Set a NoData Value

This is an important step. If you have masked pixels in your image, the output tiles will not be of equal size. To ensure each tile has the same dimensions and there are no gaps or overlapping pixels, `unmask()` all masked pixels and set a nodata value.

In [ ]:
# Assign a no-data value
noDataValue = 0
exportImage = image.unmask(**{
    'value':noDataValue,
    'sameFootprint': False
})

#### Export Tiles

We created the tiling grid using the bounding box of the region geometry. This may result in certain grids that have no overlap with the region and thus will be empty. We can filter out those empty grids before exporting.

In [ ]:
filtered_grid = grid.filter(ee.Filter.intersects('.geo', geometry))
m.addLayer(filtered_grid, {'color': 'red'}, 'Filtered Grid')
m

In [ ]:
tile_ids = filtered_grid.aggregate_array('system:index').getInfo();
print('Total tiles', len(tile_ids))

In [ ]:
# Export each tile
# Warning: This will result in 14 large GeoTIFFs tiles in your Google Drive
for i, tile_id in enumerate(tile_ids):
    feature = ee.Feature(filtered_grid.toList(1, i).get(0))
    geometry = feature.geometry()
    task_name = 'tile_' + tile_id.replace(',', '_')
    task = ee.batch.Export.image.toDrive(**{
        'image': exportImage,
        'description': f'Image_Export_{task_name}',
        'fileNamePrefix': task_name,
        'folder':'earthengine',
        'crs': crs,
        'crsTransform': transform,
        'region': geometry,
        'maxPixels': 1e10
    })
    task.start()
    print('Started Task: ', i+1)